### Metadata

In [2]:
import random
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn import model_selection
from sklearn.model_selection import cross_validate

In [3]:
# Read Training data
metadata_df = pd.read_csv("Results/meta_plus_ratings.csv")
# Read Test data
#metadata_test_df = pd.read_csv("data/Test_Set/metadata_test_plus_ratings.csv")

# Convert the runtime into int
metadata_df["runtime"] = metadata_df["runtime"].apply(lambda x: x.split(" min")[0])
metadata_df["runtime"] = metadata_df["runtime"].astype(float)

# Features
metadata_x = metadata_df.drop("goodforairplane", axis=1)
# Target
metadata_Y = metadata_df["goodforairplane"]

# Same for test set
# Features
#metadata_test_x = metadata_test_df.drop("goodforairplane", axis=1)
# Target
#metadata_test_Y = metadata_test_df["goodforairplane"]

# Map the age rating to the corresponding integer
age_mapping = {"PG-13": 13., "R": 17., "TV-MA": 18., "G": 1., "PG": 10., "NOT RATED": 0., "APPROVED": 0., np.nan: 0.}
metadata_x["rated"] = metadata_x["rated"].map(age_mapping)

# Impute the missing values of training data
values = metadata_x.iloc[:,list(range(3,7))].values
imputer = IterativeImputer(random_state=1)
transformed_values = imputer.fit_transform(values)
colnames= metadata_x.columns[3:7]
metadata_x[colnames] = transformed_values
# Impute the missing values of test data
#values = metadata_test_x.iloc[:,list(range(3,7))].values
#imputer = IterativeImputer(random_state=1)
#transformed_values = imputer.fit_transform(values)
#colnames= metadata_test_x.columns[3:7]
#metadata_test_x[colnames] = transformed_values

FileNotFoundError: [Errno 2] File b'Results/meta_plus_ratings.csv' does not exist: b'Results/meta_plus_ratings.csv'

In [11]:
np.random.seed(50)

def LVW(meta_x, meta_Y, K, classifier):
    err = 0
    k = 0
    C = 100
    # Best features
    S = np.array([])
    # Create list of features
    while k < K:
        # Create a list of available features
        features_ind = np.array(list(range(0, len(meta_x.columns))))
        num_features = len(features_ind)
        # Generate how many features should be selected
        C1 = np.random.randint(1,num_features+1)
        # Randomly pick num_selected_features as subset
        S1_ind = np.random.choice(features_ind, size=C1, replace = False)
        # Sort it, because it looks nicer...
        S1_ind = np.sort(S1_ind)
        # Train the classifier
        classifier.fit(meta_x.iloc[:,S1_ind], meta_Y)
        # Predict the results
        pred = classifier.predict(meta_x.iloc[:,S1_ind])
        # Comptue f1 measure (since in the paper it is stated that they tune it based on the f1 score)
        f1 = f1_score(meta_Y, pred)
        
        if (f1 > err or (f1 == err and C1 < C)):
            k = 0
            S = S1_ind
            err = f1
            C = C1
        else:
            k = k+1
    # Test on testset
    classifier.fit(meta_x.iloc[:,S], meta_Y)
    
    # Get column names
    columns = meta_x.iloc[:,S].columns
    return columns

In [1]:
np.random.seed(50)
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Define Scoring methods
scoring = {'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

classifiers = [KNeighborsClassifier(), NearestCentroid(), 
               DecisionTreeClassifier(), LogisticRegression(),
               SVC(), BaggingClassifier(), RandomForestClassifier(),
               AdaBoostClassifier(), GradientBoostingClassifier(), GaussianNB()]

for classi in classifiers:
    # Perform LVW (not for random forest)
    if isinstance(classi, RandomForestClassifier):
        ind = LVW(metadata_x, metadata_Y, 10, classi)
        scores = cross_validate(classi, X=metadata_x, y=metadata_Y, cv=10,scoring=scoring)
        f_1 = np.mean(scores["test_f1_score"])
        precision = np.mean(scores["test_precision"])
        recall = np.mean(scores["test_recall"])
    else:
        ind = LVW(metadata_x, metadata_Y, 10, classi)
        scores = cross_validate(classi, X=metadata_x[ind], y=metadata_Y, cv=10,scoring=scoring)
        f_1 = np.mean(scores["test_f1_score"])
        precision = np.mean(scores["test_precision"])
        recall = np.mean(scores["test_recall"])
    print("Classifier: %s, Modality: metadata, Precision: %.3f, Recall: %.3f, F1: %.3f" % (classi.__class__.__name__, precision, recall, f_1))

NameError: name 'np' is not defined